In [498]:
import numpy as np
import seaborn
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt

In [499]:
# Load the PAS questions data
df = pd.read_pickle(r'crime_data/PAS_detailed2.pkl')
df['Year-Month'] = pd.to_datetime(df['Year-Month'], format='%Y-%m-%d').dt.date

In [500]:
# lists of columns to keep in the final df
important_cols = ["Year-Month", "Year", "ward_n", "ward", "Borough"]  # w/o Week and BCU as those both start in April 2020

### Mapping PAS measures to numbers

- Q62A: Relied on to be there
- Q62E: Understand issues
- Q62TG: Listen to concerns
- Q62C: Treat everyone fairly
- NQ135BD: Trust MPS
- Q60: 'Good Job' local -- feels their local police is doing a good job
- NQ133A: Contact ward officer -- know how to contact their local ward officer
- Q131: Informed local -- feels they are informed of the local police's activities in the last 12 months

In [501]:
# Mappings for calculations
mappings = {
    'Tend to agree': 1,
    'Strongly agree': 1,
    'Neither agree nor disagree': 0,
    'Tend to disagree': 0,
    'Strongly disagree': 0
}

mappings2 = {
    'Excellent': 1,
    'Good': 1,
    'Fair': 0,
    'Poor': 0,
    'Very poor': 0
}

mappings3 = {
    'Yes': 1,
    'No': 0
}

mappings4 = {
    'Very well informed': 1,
    'Fairly well informed': 1,
    'Not at all informed': 0
}

In [502]:
# Converting measure columns for calculations
df["Q62A"] = df["Q62A"].map(mappings)
df["Q62E"] = df["Q62E"].map(mappings)
df["Q62TG"] = df["Q62TG"].map(mappings)
df["Q62C"] = df["Q62C"].map(mappings)
df["NQ135BD"] = df["NQ135BD"].map(mappings)
df["Q60"] = df["Q60"].map(mappings2)
df["NQ133A"] = df["NQ133A"].map(mappings3)
df["Q131"] = df["Q131"].map(mappings4)

## Getting the underreporting statistic for each year

Below is a list of all the questions we need to keep as well as from which years onwards they were asked.

---

We generally use the questions BQ90D and BQ90DA to determine underreporting for each year.

- BQ90D: 2015-04-01 until 2019-03-01
- BQ90DA: 2019-04-01 until 2021-03-01


In [503]:
# Time borders for BQ90D
df[["BQ90D", "Year-Month"]][~(df["BQ90D"].isna())]

,BQ90D,Year-Month
4,Yes,2015-05-01
21,No,2015-06-01
32,Yes,2015-04-01
59,Yes,2015-04-01
61,Yes,2015-05-01
...,...,...
51257,Yes,2019-02-01
51261,Yes,2019-02-01
51264,Yes,2019-03-01
51265,Yes,2019-03-01


In [504]:
# Time borders for BQ90DA
df[["BQ90D", "Year-Month"]][~(df["BQ90D"].isna())]

,BQ90D,Year-Month
4,Yes,2015-05-01
21,No,2015-06-01
32,Yes,2015-04-01
59,Yes,2015-04-01
61,Yes,2015-05-01
...,...,...
51257,Yes,2019-02-01
51261,Yes,2019-02-01
51264,Yes,2019-03-01
51265,Yes,2019-03-01


In [505]:
def calc_wards(df_func, question_column):
    # Ensure the question column is binary (1 for 'No' and 0 for 'Yes')
    if set(df_func[question_column].unique()) != {0, 1}:
        raise ValueError("Question column must be binary with 1 for 'No' and 0 for 'Yes'")

    # Group by the relevant columns and calculate the proportion of 'No' responses
    grouped = df_func.groupby(["Year-Month", "Borough", "ward_n"]).agg(
        no_count=(question_column, 'sum'),
        total_count=(question_column, 'count')
    ).reset_index()

    # Calculate the proportion of 'No' responses
    grouped["Proportion_no"] = grouped["no_count"] / grouped["total_count"]
    grouped["Total_prop_no"] = grouped["no_count"] / grouped.shape[0]
    
    # Add the question column to the result
    grouped["Question"] = question_column
    
    # Select the desired columns
    result_df = grouped[["Year-Month", "Borough", "ward_n", "Question", "total_count", "no_count", "Proportion_no", "Total_prop_no"]]
    
    return result_df



### 2015 - 2017
- BQ90A: Have you been a victim of crime or Anti-Social Behaviour in the last 12 months? => yes/no
- BQ90B: Thinking about the most recent incident, was this in your local area, elsewhere in London or outside of London?
- BQ90C: Was it...? => types of crime
- **BQ90D: Did you report this to anyone? => yes/no**
- **SQ109CAA: Who did you report it to? => _filter_**

__Extra:__
- NSQ190's: Did anything happen after you reported it?
- PQ98A: In the last 12 months has anyone tried to discourage you from reporting a crime to the police? IF YES - Who was that?
- PQ98ABA: Who was that? Police Officer
- PQ98ABB: Who was that? Another member of the police service, including someone over the phone
- PQ98B's: In what way did the police discourage you?

### 2017/18 -2018/19
__Extra:__
- XQ191's: Why did you not report this to the police?
- XQ192: If something like this happened again would you report it to the police? => yes/no  

In [506]:
print(df.shape)
print(df["BQ90A"].value_counts())
print(df["BQ90A"].value_counts()["Yes"] / df.shape[0], "respondents have experienced a crime in the past 12 months.")

(76398, 635)
BQ90A
No     67206
Yes     9140
Name: count, dtype: int64
0.11963663970261001 respondents have experienced a crime in the past 12 months.


In [507]:
# Counts for types of crime
df["BQ90C"].value_counts()

BQ90C
Property crime                                                          4037
Anti-Social Behaviour                                                   2206
Harassment                                                               754
Other                                                                    748
Violent crime                                                            533
Hate crime                                                               272
Credit/debit card fraud                                                  176
Credit/debit card fraud                                                  174
Identity theft/fraud                                                      56
Online contact that has caused fear, alarm or distress                    41
Identity theft                                                            32
Knife crime - being threatened with a knife or attacked with a knife      29
Online                                                                

In [508]:
# Counts for whether they reported the crime by types of crime
df[["BQ90C", "BQ90D"]].value_counts()

BQ90C                                                                 BQ90D
Property crime                                                        Yes      1998
Anti-Social Behaviour                                                 Yes       690
Property crime                                                        No        519
Anti-Social Behaviour                                                 No        488
Other                                                                 Yes       291
Harassment                                                            Yes       283
Violent crime                                                         Yes       257
Harassment                                                            No        141
Credit/debit card fraud                                               Yes       106
Other                                                                 No         96
Credit/debit card fraud                                               Yes        84


In [509]:
# Replace non-police entities with NaN
df = df.replace({'Residents Association': np.nan, 'Other': np.nan})
print(df["SQ109CAA"].unique())
print(df["SQ109CAA"].value_counts())  # actual reporting numbers

[nan 'Police (other than Local Policing Team)' 'Local Policing Team'
 'Local Authority/Council' 'Neighbourhood wardens']
SQ109CAA
Local Policing Team                        1781
Police (other than Local Policing Team)    1760
Local Authority/Council                     111
Neighbourhood wardens                         9
Name: count, dtype: int64


In [510]:
# add year column to check underreporting stats per year
df["Year"] = pd.DatetimeIndex(df['Year-Month']).year

In [511]:
## BQ90D asked from 2015 to start of 2019 -- only those reported to police entities
# Where they answered the question AND where it was not reported to police
df_1519 = df[important_cols + ["BQ90D"]][~(df["BQ90D"].isna()) & (df["SQ109CAA"].isna())].copy()

df_1519["BQ90D"] = df_1519["BQ90D"].map({
    'Yes': 0,
    'No': 1
})

# Total underreporting 2015 - 2019
print(df_1519.shape)
print(df_1519[["BQ90D"]].value_counts())
print("Percentage of underreporting 2015-2019: ", df_1519[df_1519["BQ90D"] == 1.0].shape[0]/df_1519.shape[0])
print("-------------------------------------------------------")

for year in df_1519["Year"].unique():
    df_15_insights = df_1519[df_1519["Year"] == year]
    print(f"% total underreporting {year}: ", df_15_insights[df_15_insights["BQ90D"] == 1.0].shape[0]/df[df["Year"] == year].shape[0])
print("2019 underreporting goes until the end of February 2019.")

df2 = calc_wards(df_1519, "BQ90D")
df2

(1636, 6)
BQ90D
1        1411
0         225
Name: count, dtype: int64
Percentage of underreporting 2015-2019:  0.8624694376528117
-------------------------------------------------------
% total underreporting 2015:  0.0265974025974026
% total underreporting 2016:  0.02523196048327882
% total underreporting 2017:  0.02800936768149883
% total underreporting 2018:  0.0280986575085857
% total underreporting 2019:  0.009210834439153852
2019 underreporting goes until the end of February 2019.


,Year-Month,Borough,ward_n,Question,total_count,no_count,Proportion_no,Total_prop_no
0,2015-04-01,Barking and Dagenham,Heath,BQ90D,1,1,1.0,0.000683
1,2015-04-01,Bexley,Barnehurst,BQ90D,1,1,1.0,0.000683
2,2015-04-01,Brent,Barnhill,BQ90D,1,0,0.0,0.000000
3,2015-04-01,Brent,Kenton,BQ90D,1,1,1.0,0.000683
4,2015-04-01,Brent,Kilburn,BQ90D,1,1,1.0,0.000683
...,...,...,...,...,...,...,...,...
1459,2019-03-01,Sutton,Belmont,BQ90D,1,1,1.0,0.000683
1460,2019-03-01,Waltham Forest,Lea Bridge,BQ90D,1,1,1.0,0.000683
1461,2019-03-01,Wandsworth,Thamesfield,BQ90D,1,1,1.0,0.000683
1462,2019-03-01,Westminster,Church Street,BQ90D,1,1,1.0,0.000683


### 2019/20 - 2020/21

- **BQ90DA: Did you report it to the police? => yes/no**

__Extra:__
- XQ119B: As a result of your contact with the police on this occasion, please tell me if your opinion is now better, worse or has not changed?

In [512]:
## BQ90DA asked from 2019 to start of 2021 -- specifically for the police
df_1921 = df[important_cols + ["BQ90DA"]][~(df["BQ90DA"].isna())].copy()

df_1921["BQ90DA"] = df_1921["BQ90DA"].map({
    'Yes': 0,
    'No': 1
})

# Total underreporting 2019 - 2021
print(df_1921.shape)
print(df_1921[["BQ90DA"]].value_counts())
print("Percentage of underreporting 2019-2021: ", df_1921[df_1921["BQ90DA"] == 1.0].shape[0]/df_1921.shape[0])
print("-------------------------------------------------------")

for year in df_1921["Year"].unique():
    df_19_insights = df_1921[df_1921["Year"] == year]
    print(f"% total underreporting {year}: ", df_19_insights[df_19_insights["BQ90DA"] == 1.0].shape[0]/df[df["Year"] == year].shape[0])
print("2021 underreporting goes until the end of February 2021.")

df3 = calc_wards(df_1921, "BQ90DA")
df3

(3835, 6)
BQ90DA
0         2682
1         1153
Name: count, dtype: int64
Percentage of underreporting 2019-2021:  0.30065189048239893
-------------------------------------------------------
% total underreporting 2019:  0.03130122550932792
% total underreporting 2020:  0.049059953199386754
% total underreporting 2021:  0.04643663334408255
2021 underreporting goes until the end of February 2021.


,Year-Month,Borough,ward_n,Question,total_count,no_count,Proportion_no,Total_prop_no
0,2019-04-01,#N/A,Hobbayne,BQ90DA,1,0,0.0,0.000000
1,2019-04-01,#N/A,St George's,BQ90DA,1,0,0.0,0.000000
2,2019-04-01,Barking and Dagenham,Goresbrook,BQ90DA,1,0,0.0,0.000000
3,2019-04-01,Barking and Dagenham,Valence,BQ90DA,1,1,1.0,0.000323
4,2019-04-01,Barnet,Coppetts,BQ90DA,1,0,0.0,0.000000
...,...,...,...,...,...,...,...,...
3092,2021-03-01,Wandsworth,Fairfield,BQ90DA,1,0,0.0,0.000000
3093,2021-03-01,Wandsworth,Thamesfield,BQ90DA,1,1,1.0,0.000323
3094,2021-03-01,Westminster,Hyde Park,BQ90DA,1,1,1.0,0.000323
3095,2021-03-01,Westminster,West End,BQ90DA,1,0,0.0,0.000000


### 2021/2022

**NOTE:** This specific year does not have BQ90D(A), hence we need to take weighted averages of the negative answers of the following:

- XQ59C: How likely or unlikely would you be to report the following to the police...? Someone you know personally, who you think might be involved in terrorism
- XQ59D: How likely or unlikely would you be to report the following to the police...? Someone you know personally, who you think might be vulnerable to radicalisation

THIS YEAR AND ONWARDS WILL LIKELY BE DISCARDED WHEN CALCULATING UNDERREPORTING DUE TO TOO SPECIFIC QUESTIONS.

In [513]:
# Mapping scales to interpretable values -- possible underreporting
df["XQ59C"] = df["XQ59C"].map({
    'Very likely': 0,
    'Fairly likely': 0,
    'Neither likely nor unlikely': 0,
    'Fairly unlikely': 1,
    'Very unlikely': 1,
    'Refused': np.nan
})

df["XQ59D"] = df["XQ59D"].map({
    'Very likely': 0,
    'Fairly likely': 0,
    'Neither likely nor unlikely': 0,
    'Fairly unlikely': 1,
    'Very unlikely': 1,
    'Refused': np.nan
})

In [514]:
# Total underreporting 2021 - 2022 -- difficult questions that don't cover all of underreporting (only on terrorism and radicalization)
df_2122 = df[important_cols + ["XQ59C", "XQ59D"]][~((df["XQ59C"].isna()) | (df["XQ59D"].isna()))].copy()

print(df_2122.shape)
print(df_2122[["XQ59C"]].value_counts())
print("Percentage of possible underreporting terrorism 2021-2022: ", df_2122[df_2122["XQ59C"] == 1.0].shape[0]/df_2122.shape[0])
print(df_2122[["XQ59D"]].value_counts())
print("Percentage of possible underreporting radicalization 2021-2022: ", df_2122[df_2122["XQ59D"] == 1.0].shape[0]/df_2122.shape[0])

(21142, 7)
XQ59C
0.0      20296
1.0        846
Name: count, dtype: int64
Percentage of possible underreporting terrorism 2021-2022:  0.04001513574874657
XQ59D
0.0      20028
1.0       1114
Name: count, dtype: int64
Percentage of possible underreporting radicalization 2021-2022:  0.05269132532399962


In [515]:
# Combining cells
df_2122 = df_2122.astype({"XQ59C": int, "XQ59D": int})
df_2122["XQ59CD"] = df_2122["XQ59C"] | df_2122["XQ59D"]
df4 = calc_wards(df_2122, "XQ59CD")
df4

,Year-Month,Borough,ward_n,Question,total_count,no_count,Proportion_no,Total_prop_no
0,2018-07-01,,Blackwall & Cubitt Town,XQ59CD,2,0,0.0,0.0
1,2018-07-01,,Chapel End,XQ59CD,1,0,0.0,0.0
2,2018-07-01,,Colindale,XQ59CD,1,0,0.0,0.0
3,2018-07-01,,Greenwich West,XQ59CD,1,0,0.0,0.0
4,2018-07-01,,Junction,XQ59CD,1,0,0.0,0.0
...,...,...,...,...,...,...,...,...
4566,2020-03-01,Westminster,Harrow Road,XQ59CD,8,0,0.0,0.0
4567,2020-03-01,Westminster,Knightsbridge and Belgravia,XQ59CD,2,0,0.0,0.0
4568,2020-03-01,Westminster,Lancaster Gate,XQ59CD,1,0,0.0,0.0
4569,2020-03-01,Westminster,Warwick,XQ59CD,2,0,0.0,0.0


### 2022/2023

**NOTE:** This specific year does not have BQ90D(A). Only this question was available for underreporting:

- **NQ1I: To what extent do you agree or disagree with the following statements: I would report a rape to the police if it happened to me.**

__Extra:__
- NQ1A: To what extent do you agree or disagree with the following statements: The police take reports of rape seriously.
- Q201H: To what extent do you agree or disagree with the following statements: The police treat men who report that they have been raped in the same way as they would treat a female complainant.



In [516]:
# Final underreporting dataframe
df_fin = pd.concat([df2, df3, df4], ignore_index=True)
print(df_fin)

# df_fin.to_pickle(r"PAS_underreporting.pkl")

      Year-Month               Borough                       ward_n Question   
0     2015-04-01  Barking and Dagenham                        Heath    BQ90D  \
1     2015-04-01                Bexley                   Barnehurst    BQ90D   
2     2015-04-01                 Brent                     Barnhill    BQ90D   
3     2015-04-01                 Brent                       Kenton    BQ90D   
4     2015-04-01                 Brent                      Kilburn    BQ90D   
...          ...                   ...                          ...      ...   
9127  2020-03-01           Westminster                  Harrow Road   XQ59CD   
9128  2020-03-01           Westminster  Knightsbridge and Belgravia   XQ59CD   
9129  2020-03-01           Westminster               Lancaster Gate   XQ59CD   
9130  2020-03-01           Westminster                      Warwick   XQ59CD   
9131  2020-03-01           Westminster                   Westbourne   XQ59CD   

      total_count  no_count  Proportion

## Underreporting along with trust

What does trust look like for those that did not report the crimes they experienced?

In [521]:
df_fin

,Year-Month,Borough,ward_n,Question,total_count,no_count,Proportion_no,Total_prop_no
0,2015-04-01,Barking and Dagenham,Heath,BQ90D,1,1,1.0,0.000683
1,2015-04-01,Bexley,Barnehurst,BQ90D,1,1,1.0,0.000683
2,2015-04-01,Brent,Barnhill,BQ90D,1,0,0.0,0.000000
3,2015-04-01,Brent,Kenton,BQ90D,1,1,1.0,0.000683
4,2015-04-01,Brent,Kilburn,BQ90D,1,1,1.0,0.000683
...,...,...,...,...,...,...,...,...
9127,2020-03-01,Westminster,Harrow Road,XQ59CD,8,0,0.0,0.000000
9128,2020-03-01,Westminster,Knightsbridge and Belgravia,XQ59CD,2,0,0.0,0.000000
9129,2020-03-01,Westminster,Lancaster Gate,XQ59CD,1,0,0.0,0.000000
9130,2020-03-01,Westminster,Warwick,XQ59CD,2,0,0.0,0.000000


## Look into crime per borough per year

In [517]:
# # Load data with preprocessing
# df = pd.read_pickle(r'crime_data/london_street_with_city.pkl')
# df = df[~(df["Borough"] == 'City of London')]
# df["Year"] = df["Year"].astype(int)
# df["Year-Month"] = pd.to_datetime(df["Year-Month"]).dt.date
# df = df[(df["Year"] >= 2014) & (df["Year"] <= 2019)]

In [518]:
# # Grouping by crime
# crime_counts = df.groupby(['Year-Month', 'Crime type']).size().reset_index(name='Count')
#
# # Pivot table to convert Crime type into columns
# pivot_df = crime_counts.pivot_table(index='Year-Month', columns='Crime type', values='Count', aggfunc='sum', fill_value=0)
# print(pivot_df.head(10))

In [519]:
# def plotter(pivot_df):
#     plt.figure(figsize=(12, 6))
#     plt.stackplot(pivot_df.index, pivot_df.values.T, labels=pivot_df.columns)
#     plt.title('Crime Counts Over Time')
#     plt.xlabel('Year-Month')
#     plt.ylabel('Count')
#     plt.legend(loc='upper left')
#     plt.xticks(rotation=45)
#     plt.tight_layout()
#     plt.show()

In [520]:
# plotter(pivot_df)